In [129]:
import numpy as np
import cv2
import mediapipe as mp
import time
import tkinter as tk
from PIL import Image, ImageTk

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.1, model_complexity=2)

def calculate_angles(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180:
        angle=360-angle
    return round(angle,2)

# Open the camera
cap = cv2.VideoCapture(0)
start_time = time.time()

# Initialize Tkinter window
root = tk.Toplevel()
root.title("Pose Detection")
screen_width=root.winfo_screenwidth()
screen_height=root.winfo_screenheight()
root.geometry(f"{screen_width}x{screen_height}")

text_label = tk.Label(root, text="Yoga Pose Correction",font=("Times New Roman",50))
text_label.place(x=340, y=10)


correct_images="pranamasana.png"
correct_images=image_result(correct_images)
correct_images="image.png"
correct_image=Image.open(correct_images)
correct_image_tk=ImageTk.PhotoImage(correct_image)

image_label=tk.Label(root,image=correct_image_tk)
image_label.place(x=200,y=140)

canvas = tk.Canvas(root, width=640, height=480)
canvas.place(x=600,y=100)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    try:
        landmarks = results.pose_landmarks.landmark
        landmarks=results.pose_landmarks.landmark

        left_shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        angle=calculate_angles(left_shoulder,left_elbow,left_wrist)
        cv2.putText(image, f"{angle}", (int(left_elbow[0] * image.shape[1]), int(left_elbow[1] * image.shape[0])),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.1, (300, 300, 300), 3, cv2.LINE_AA)
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

        angle = calculate_angles(right_shoulder, right_elbow, right_wrist)

        cv2.putText(image, f"{angle}", (int(right_elbow[0] * image.shape[1]-50), int(right_elbow[1] * image.shape[0])),
            cv2.FONT_HERSHEY_SIMPLEX, 1.1, (300, 300, 300), 3, cv2.LINE_AA)
       
    except:
        pass

    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(232, 231, 21), thickness=2, circle_radius=3),
        mp_drawing.DrawingSpec(color=(0, 230, 0), thickness=1)
    )

    # Convert image to ImageTk format for Tkinter display
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    img = Image.fromarray(image)
    img_tk = ImageTk.PhotoImage(image=img)
    
    canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
    root.update()

    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
    current_time = time.time()
    if current_time - start_time >= 5:
        screenshot = image.copy()  # Capture the frame directly
        cv2.imwrite("screenshot.png", screenshot)
        break

cap.release()
cv2.destroyAllWindows()
root.mainloop()


In [118]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.1, model_complexity=2)

def calculate_angles(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180:
        angle=360-angle
    return round(angle,2)

def image_result(image):
    image=cv2.imread(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image to BGR format
    image.flags.writeable=False
    results = pose.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    try:
        landmarks=results.pose_landmarks.landmark
        left_shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        angle=calculate_angles(left_shoulder,left_elbow,left_wrist)
        cv2.putText(image, f"{angle}", (int(left_elbow[0] * image.shape[1])-20, int(left_elbow[1] * image.shape[0])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (300, 300, 300), 1, cv2.LINE_AA)
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        angle = calculate_angles(right_shoulder, right_elbow, right_wrist)
        cv2.putText(image, f"{angle}", (int(right_elbow[0] * image.shape[1])-30, int(right_elbow[1] * image.shape[0])),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (300, 300, 300), 1, cv2.LINE_AA)
    except:
        pass
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(100, 100, 100), thickness=2, circle_radius=1),
        mp_drawing.DrawingSpec(color=(100, 0, 0), thickness=1)
    )

    return cv2.imwrite("image.png",image)
#     cv2.imshow("image", image)
#     cv2.waitKey(0) & 0xFF==ord("q")
#     cv2.destroyAllWindows()

# image_path = "pranamasana.png"
# loaded_image = cv2.imread(image_path)

# image_result(loaded_image)


In [95]:
tk.Label?

Init signature: tk.Label(master=None, cnf={}, **kw)
Docstring:      Label widget which can display text and bitmaps.
Init docstring:
Construct a label widget with the parent MASTER.

STANDARD OPTIONS

    activebackground, activeforeground, anchor,
    background, bitmap, borderwidth, cursor,
    disabledforeground, font, foreground,
    highlightbackground, highlightcolor,
    highlightthickness, image, justify,
    padx, pady, relief, takefocus, text,
    textvariable, underline, wraplength

WIDGET-SPECIFIC OPTIONS

    height, state, width
File:           c:\users\25bak\anaconda3\lib\tkinter\__init__.py
Type:           type
Subclasses:     


In [3]:
def calculate_angles(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)
    if angle>180:
        angle=360-angle
    return round(angle,2)

In [80]:
import numpy as np
import cv2
import mediapipe as mp
from PIL import Image

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.1, model_complexity=2)

def calculate_angles(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return round(angle, 2)

def image_result(image, correct_image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert image to BGR format
    image.flags.writeable = False
    results = pose.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    try:
        landmarks = results.pose_landmarks.landmark
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        angle = calculate_angles(left_shoulder, left_elbow, left_wrist)
        cv2.putText(image, f"{angle}", (int(left_elbow[0] * image.shape[1]) - 20,
                                         int(left_elbow[1] * image.shape[0])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 0, 0), 1, cv2.LINE_AA)
        
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        angle = calculate_angles(right_shoulder, right_elbow, right_wrist)
        cv2.putText(image, f"{angle}", (int(right_elbow[0] * image.shape[1]) - 30,
                                         int(right_elbow[1] * image.shape[0])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 0, 0), 1, cv2.LINE_AA)
    except:
        pass

    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(200, 0, 0), thickness=2, circle_radius=1),
        mp_drawing.DrawingSpec(color=(200, 0, 0), thickness=1)
    )
    
    # Display both the corrected image and the pose landmarks
    correct_image = correct_image.resize((image.shape[1], image.shape[0]))
    correct_image = cv2.cvtColor(np.array(correct_image), cv2.COLOR_RGB2BGR)
    combined_image = cv2.addWeighted(image, 0.5, correct_image, 0.5, 0)
    
    cv2.imshow("Image with Pose Landmarks and Corrected Image", combined_image)
    cv2.waitKey(0) & 0xFF == ord("q")
    cv2.destroyAllWindows()

# Load the correct image
correct_image_path = "pranamasana.png"
correct_image = Image.open(correct_image_path)

# Open the camera
cap = cv2.VideoCapture(0)
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    image_result(frame, correct_image)

    current_time = time.time()
    if current_time - start_time >= 5:
        screenshot = frame.copy()  # Capture the frame directly
        cv2.imwrite("screenshot.png", screenshot)
        break

cap.release()
cv2.destroyAllWindows()
